In [23]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from skimage.feature import hog

In [24]:
good_dir = r'C:\Users\USER\Pictures\Nadun\Project\Good'
broken_dir = r'C:\Users\USER\Pictures\Nadun\Project\Bad'

images = []
labels = []

In [25]:
def load_images_from_folder(folder, label):
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (150, 150))
            images.append(img)
            labels.append(label)

load_images_from_folder(good_dir, 'good')
load_images_from_folder(broken_dir, 'broken')

In [26]:
images = np.array(images)
labels = np.array(labels)

In [27]:
le = LabelEncoder()
labels = le.fit_transform(labels)

In [28]:
hog_features = []
for image in images:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray_image, pixels_per_cell=(64, 64),
                      cells_per_block=(2, 2), visualize=True)
    hog_features.append(features)

hog_features = np.array(hog_features)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)


In [30]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [31]:
# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Model accuracy: {accuracy*100:.2f}%")


Random Forest Model accuracy: 100.00%


In [33]:
def predict_new_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (150, 150))
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray_image, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), visualize=True)
    features = features.reshape(1, -1)
    prediction = rf_model.predict(features)
    label = le.inverse_transform(prediction)
    return label[0]

# Example prediction
new_image_path = r'C:\Users\USER\Pictures\Nadun\Project\Broken-test.jpg'
print(f"The image is predicted as: {predict_new_image(new_image_path)}")

ValueError: X has 10404 features, but RandomForestClassifier is expecting 36 features as input.